In [ ]:

import json
import networkx as nx
from pyvis.network import Network
import webbrowser
from pathlib import Path

# ------------------- 1. LOAD YOUR DATA -------------------
FILE_PATH = r"D:\python\got\v2\data_done\relations\all_books_relations_merged.json"

print("Loading your massive dataset...")
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    relationships = json.load(f)

print(f"Loaded {len(relationships):,} relationships → {len({r['from'] for r in relationships} | {r['to'] for r in relationships}):,} characters")

# ------------------- 2. BUILD GRAPH -------------------
G = nx.DiGraph()
for r in relationships:
    fr = r["from"].strip()
    to = r["to"].strip()
    rel = r.get("relationship_type", "related").lower()
    details = r.get("key_details", "").strip()
    
    G.add_edge(fr, to, label=rel.replace("_", " "), title=details or rel, type=rel)

print(f"Graph built: {G.number_of_nodes():,} nodes • {G.number_of_edges():,} edges")

# ------------------- 3. CREATE THE PERFECT HTML GRAPH -------------------
net = Network(
    height="100vh",
    width="100%",
    directed=True,
    bgcolor="#0a0a0a",
    font_color="#ffffff",
    select_menu=True,
    filter_menu=True,
    notebook=False
)

# Use BarnesHut – best for large graphs
net.options = {
    "physics": {
        "enabled": True,
        "barnesHut": {
            "gravitationalConstant": -8000,
            "centralGravity": 0.3,
            "springLength": 150,
            "springConstant": 0.04,
            "damping": 0.9,
            "avoidOverlap": 1
        },
        "maxVelocity": 146,
        "solver": "barnesHut",
        "stabilization": {
            "enabled": True,
            "iterations": 2000
        }
    },
    "interaction": {
        "hover": True,
        "navigationButtons": True,
        "keyboard": True
    },
    "edges": {
        "arrows": "to",
        "smooth": True,
        "color": "#555555"
    }
}

# Add ALL nodes
for node in G.nodes():
    color = "#95a5a6"
    name = node.upper()
    if any(h in name for h in ["STARK", "NED", "ROBB", "ARYA", "SANSA", "LYANNA", "BRANDON"]):
        color = "#3498db"
    elif any(h in name for h in ["LANNISTER", "TYWIN", "CERSEI", "JAIME", "TYRION"]):
        color = "#f1c40f"
    elif any(h in name for h in ["TARGARYEN", "DAENERYS", "RHAEGAR", "AERYS"]):
        color = "#e74c3c"
    elif "BARATHEON" in name:
        color = "#e67e22"
    elif "TYRELL" in name:
        color = "#27ae60"
    elif "MARTELL" in name:
        color = "#d35400"
    elif "GREYJOY" in name:
        color = "#2c3e50"
    
    net.add_node(node, label=node, title=node, color=color, size=28)

# Add ALL edges
for u, v, data in G.edges(data=True):
    net.add_edge(u, v, title=data["title"], arrows="to", width=2)

# ------------------- 4. ADD MAGIC CLICK-TO-ISOLATE SCRIPT -------------------
html = net.generate_html()

# Inject the perfect JavaScript
magic_script = """
<script type="text/javascript">
    // Click a node → show only its neighbors
    network.on("click", function(params) {
        if (params.nodes.length === 1) {
            var nodeId = params.nodes[0];
            var connected = network.getConnectedNodes(nodeId);
            var allNodes = network.body.data.nodes.getIds();
            
            allNodes.forEach(function(n) {
                if (n !== nodeId && !connected.includes(n)) {
                    network.body.data.nodes.update({id: n, hidden: true});
                } else {
                    network.body.data.nodes.update({id: n, hidden: false, size: n === nodeId ? 50 : 35});
                }
            });
        }
    });

    // Double-click → show everything again
    network.on("doubleClick", function() {
        network.body.data.nodes.update(
            network.body.data.nodes.get().map(n => ({id: n.id, hidden: false, size: 28}))
        );
    });

    // ESC key → fit to screen
    document.addEventListener('keydown', function(e) {
        if (e.key === 'Escape') network.fit();
    });
</script>
"""

final_html = html.replace("</body>", magic_script + "</body>")

# ------------------- 5. SAVE & OPEN -------------------
output_file = Path("GOT_KNOWLEDGE_GRAPH_PERFECT.html")
with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_html)



webbrowser.open(f"file://{output_file.resolve()}")



Loading your massive dataset...
Loaded 35,229 relationships → 3,646 characters
Graph built: 3,646 nodes • 6,713 edges

YOUR MASTERPIECE IS READY!
File: GOT_KNOWLEDGE_GRAPH_PERFECT.html
   • All 3,646 characters
   • Click any node → isolates family & allies
   • Double-click → show full map
   • ESC → zoom to fit
   • Search, filter, drag, zoom — everything works perfectly

Opening in your browser now...
You are now the true Grand Maester of Westeros.


In [4]:
import json
import networkx as nx
from pyvis.network import Network
import webbrowser
from pathlib import Path
import re

# ------------------- LOAD DATA -------------------
FILE_PATH = r"D:\python\got\v2\data_done\relations\all_books_relations_merged.json"
print("Loading ASOIAF relationships...")
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    relationships = json.load(f)

G = nx.DiGraph()
for r in relationships:
    fr = re.sub(r'\s+', ' ', str(r.get("from", "")).strip()).title()
    to = re.sub(r'\s+', ' ', str(r.get("to", "")).strip()).title()
    if not fr or not to: continue
    rel = str(r.get("relationship_type", "related")).title().replace("_", " ")
    details = str(r.get("key_details", "")).strip()
    G.add_edge(fr, to, title=details or rel, label=rel)

print(f"Graph built: {G.number_of_nodes():,} nodes | {G.number_of_edges():,} edges")

# ------------------- OUTPUT -------------------
output_folder = Path("GOT_MINIMAL_PERFECT")
output_folder.mkdir(exist_ok=True)

# ------------------- HOUSE COLORS -------------------
def get_house_color(name):
    n = name.upper()
    if any(x in n for x in ["STARK", "NED", "ROBB", "ARYA", "SANSA", "JON", "BRAN"]):     return "#3498db"
    if any(x in n for x in ["LANNISTER", "TYWIN", "CERSEI", "JAIME", "TYRION"]):        return "#f1c40f"
    if any(x in n for x in ["TARGARYEN", "DAENERYS", "RHAEGAR", "AERYS", "VISERYS"]):   return "#e74c3c"
    if any(x in n for x in ["BARATHEON", "ROBERT", "STANNIS", "RENLY"]):               return "#e67e22"
    if "TYRELL" in n or "MARGAERY" in n:                                                return "#27ae60"
    if "MARTELL" in n or "OBERYN" in n:                                                 return "#d35400"
    if "GREYJOY" in n or "THEON" in n:                                                  return "#2c3e50"
    if "BOLTON" in n or "RAMSAY" in n:                                                  return "#c0392b"
    return "#95a5a6"

# ------------------- FINAL CLEAN SCRIPT -------------------
magic_script = """
<script type="text/javascript">
    const search = document.createElement("input");
    search.type = "text";
    search.placeholder = "Search... (e.g. Jon, Arya, Killed)";
    search.style.cssText = "position:fixed;top:10px;left:50%;transform:translateX(-50%);z-index:9999;padding:10px 18px;width:400px;font-size:16px;border:none;border-radius:30px;outline:none;background:#000;color:#fff;box-shadow:0 4px 15px rgba(0,0,0,0.6);text-align:center;";
    document.body.appendChild(search);

    search.addEventListener("input", function() {
        const term = this.value.toLowerCase().trim();
        const nodes = network.body.data.nodes;
        nodes.getIds().forEach(id => {
            const label = (nodes.get(id).label || "").toLowerCase();
            nodes.update({id: id, hidden: term && !label.includes(term)});
        });
    });

    network.on("click", p => {
        if (p.nodes.length === 1) {
            const node = p.nodes[0];
            const conn = network.getConnectedNodes(node);
            network.body.data.nodes.getIds().forEach(n => {
                const visible = n === node || conn.includes(n);
                network.body.data.nodes.update({id: n, hidden: !visible, size: visible ? (n===node ? 40 : 18) : 14});
            });
        }
    });

    network.on("doubleClick", () => {
        network.body.data.nodes.update(network.body.data.nodes.get().map(n => ({id: n.id, hidden: false, size: 14})));
        search.value = "";
        network.fit({animation: false});
    });

    document.addEventListener("keydown", e => {
        if (e.key === "Escape") network.fit({animation: false});
        if (e.key === "r") location.reload();
    });

    setTimeout(() => network.fit({animation: false}), 500);
</script>
"""

# ------------------- GENERATE MINIMAL PERFECT GRAPHS -------------------
threshold = 21
major_chars = sorted([n for n, d in G.degree() if d >= threshold], key=lambda x: G.degree(x), reverse=True)

print(f"\nGenerating {len(major_chars)} MINIMAL PERFECT graphs with VISIBLE RELATIONS...\n")

for i, center in enumerate(major_chars, 1):
    print(f"  [{i:02d}] {center} ({G.degree(center)} connections)")

    neighbors = set(G.neighbors(center)) | set(G.predecessors(center))
    nodes = {center} | neighbors
    H = G.subgraph(nodes).copy()

    net = Network(height="100vh", width="100%", directed=True, bgcolor="#0a0a0a", font_color="#ddd")
    net.from_nx(H)
    net.toggle_physics(False)

    # CLEAN, MINIMAL, PERFECT OPTIONS
    net.set_options('''
    {
        "nodes": {
            "shape": "dot",
            "size": 14,
            "font": {"size": 13, "face": "arial", "strokeWidth": 3, "strokeColor": "#000000"},
            "borderWidth": 2,
            "shadow": true
        },
        "edges": {
            "arrows": {"to": {"enabled": true, "scaleFactor": 0.5}},
            "smooth": false,
            "font": {
                "size": 10,
                "align": "middle",
                "color": "#fff",
                "strokeWidth": 2,
                "strokeColor": "#000000",
                "background": "rgba(10,10,10,0.7)",
                "padding": 2
            },
            "color": {"color": "#666666", "highlight": "#999999"},
            "width": 1.5,
            "arrowStrikethrough": false
        },
        "interaction": {
            "hover": true,
            "navigationButtons": false,
            "keyboard": false
        },
        "layout": {"randomSeed": 123}
    }
    ''')

    # NODES — TINY 14px
    for node in net.nodes:
        name = node["id"]
        node["size"] = 40 if name == center else 14
        node["color"] = {"background": get_house_color(name), "border": "#ffffff", "highlight": {"border": "#ffff00"}}
        node["font"] = {"size": 28 if name == center else 13}

    # CENTER NODE — KING
    center_node = net.get_node(center)
    if center_node:
        center_node.update({
            "size": 24,
            "font": {"size": 32, "strokeWidth": 10},
            "borderWidth": 5,
            "color": {"background": get_house_color(center), "border": "#ffffff"}
        })

    # EDGES — ALWAYS SHOW RELATIONSHIP LABEL
    for edge in net.edges:
        rel_label = H.edges[edge["from"], edge["to"]].get("label", "")
        details = H.edges[edge["from"], edge["to"]].get("title", "")

        edge["label"] = rel_label
        edge["title"] = f"<b>{rel_label}</b><br><small>{details}</small>" if details else rel_label
        edge["font"] = {"size": 11, "color": "#ffffff", "background": "rgba(0,0,0,0.6)"}

        if edge["from"] == center or edge["to"] == center:
            edge["width"] = 3
            edge["color"] = {"color": "#ff6b6b"}
            edge["font"]["size"] = 13
            edge["font"]["color"] = "#ff9999"

    # Generate & save
    html = net.generate_html()
    html = html.replace("</body>", magic_script + "</body>")

    safe_name = re.sub(r'[<>:"/\\|?*]', '_', center)
    filename = output_folder / f"{i:02d}_{safe_name}_MINIMAL.html"
    filename.write_text(html, encoding="utf-8")

print(f"\nABSOLUTE MASTERPIECE COMPLETE!")
print(f"{len(major_chars)} tiny, perfect, readable graphs saved!")
print(f"→ {output_folder.resolve()}\n")

# Open first
try:
    webbrowser.open(f"file://{(output_folder / '01_Tyrion Lannister_MINIMAL.html').resolve()}")
except:
    webbrowser.open(f"file://{(next(output_folder.iterdir())).resolve()}")

Loading ASOIAF relationships...
Graph built: 2,896 nodes | 5,621 edges

Generating 58 MINIMAL PERFECT graphs with VISIBLE RELATIONS...

  [01] Their Children (104 connections)
  [02] Unknown (88 connections)
  [03] —Their Children (77 connections)
  [04] Stannis Baratheon (73 connections)
  [05] —His Siblings (72 connections)
  [06] Robb Stark (71 connections)
  [07] Balon Greyjoy (65 connections)
  [08] Mace Tyrell (60 connections)
  [09] Night’S Watch (60 connections)
  [10] His Household (58 connections)
  [11] Daenerys Targaryen (56 connections)
  [12] Joffrey Baratheon (53 connections)
  [13] Night'S Watch (53 connections)
  [14] The Queen (51 connections)
  [15] Euron Greyjoy (48 connections)
  [16] Tywin Lannister (46 connections)
  [17] Renly Baratheon (45 connections)
  [18] —His Household (44 connections)
  [19] Unspecified Patriarch (44 connections)
  [20] Lord Walder Frey (43 connections)
  [21] Walder Frey (41 connections)
  [22] Doran Nymeros Martell (39 connections)
  [2